In [2]:
import dask.distributed
import pystac_client
import stackstac
import dask.array as da
from geogif import gif, dgif
import geopandas as gpd

#Load STAC items into xarray Datasets. Process locally or distribute data loading and computation with Dask.
from odc.stac import configure_rio, load
from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry

In [3]:
client = dask.distributed.Client()
display(client)

2022-10-18 12:04:14,800 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ygyc5hx0', purging
2022-10-18 12:04:14,932 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-pb0lnh_e', purging
2022-10-18 12:04:15,091 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wg6bykyt', purging
2022-10-18 12:04:15,237 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-a_zpibk_', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 24.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43807,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 24.77 GiB
Comm: tcp://127.0.0.1:46045,Total threads: 3
Dashboard: http://127.0.0.1:34505/status,Memory: 6.19 GiB
Nanny: tcp://127.0.0.1:36997,


In [4]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [5]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [6]:
%%time

bbox =aoi.unary_union.bounds
items = (
    pystac_client.Client.open("https://earth-search.aws.element84.com/v0")
    .search(
        bbox=bbox,
        collections=["sentinel-s2-l2a-cogs"],
        query={"eo:cloud_cover":{"lt":1},"sentinel:valid_cloud_cover": {"eq": True}},
        datetime = "2016-05-01/2022-06-30"
    )
)

items.matched()

CPU times: user 106 ms, sys: 8.1 ms, total: 114 ms
Wall time: 1.19 s


1113

Turn STAC items into xarray as a temporal stack, using stackstac.


In [7]:
stack = stackstac.stack(items.item_collection(), bounds_latlon=bbox, epsg = 32620, resolution=30)

In [8]:
type (stack)

xarray.core.dataarray.DataArray


Then mask out bad (cloudy) pixels, according to the Sentinel-2 SCL Scene Classification Map, and take the temporal median of each quarter (three months) to hopefully get an okay-looking cloud-free frame representative of those three months.

In [ ]:

scl = stack.sel(band=["SCL"])
# Sentinel-2 Scene Classification Map: nodata, saturated/defective, dark, cloud shadow, cloud med. prob., cloud high prob., cirrus
invalid = da.isin(scl, [0, 1, 2, 3, 8, 9, 10])
valid = stack.where(~invalid)

rgb = valid.sel(band=["B04", "B03", "B02"])

quarterly = rgb.resample(time="Q").median()
quarterly

In [ ]:
#ts = quarterly.persist()
#ts_local = quarterly.compute()
#ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [ ]:
ts = quarterly.persist()
cleaned = ts[2:].ffill("time").bfill("time")

In [ ]:
gif_data = dgif(cleaned, bytes=True).compute()


In [ ]:
#dgif(cleaned).compute()


In [ ]:
with open("petermann_1sem2020.gif", "wb") as f:
    f.write(gif_data)